In [5]:
import pandas as pd
import datetime
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import os
import math
import numpy as np
import canada_holiday
import joblib  # To load the scaler
from sklearn.feature_selection import mutual_info_regression


import seaborn as sns
from sklearn import preprocessing

from pyhelpers.store import save_fig, save_svg_as_emf
import subprocess
import  aspose.cells 
from aspose.cells import Workbook

In [6]:
#%% Student directory
hanad_run = ["./data", 1]
clover_run = ["./data", 2]
joseph_laptop_run = ["C:\\Users\\sposa\\Documents\\GitHub\\power-forecasting-capstone\\data", 3]
joseph_pc_run = ["D:\\Users\\Joseph\\Documents\\GitHub\\power-forecasting-capstone\\data", 3]
janna_run = ["../data", 4]

###############################################################################
############### MAKE SURE TO CHANGE BEFORE RUNNING CODE #######################
###############################################################################
# Paste student name_run for whoever is running the code
run_student = janna_run
if (run_student[1] == joseph_laptop_run[1]):
    print("JOSEPH IS RUNNING!")
elif (run_student[1] == hanad_run[1]):
    print("HANAD IS RUNNING!")
elif (run_student[1] == janna_run[1]):
    print("JANNA IS RUNNING!")
elif (run_student[1] == clover_run[1]):
    print("CLOVER IS RUNNING!")
else:
    print("ERROR!! NO ELIGIBLE STUDENT!")
            
#%% User input

# FSA - Forward Section Area (first 3 characters of Postal Code)
#       L8K = Neighborhood in Hamilton (Link to FSA Map: https://www.prospectsinfluential.com/wp-content/uploads/2017/09/Interactive-Canadian-FSA-Map.pdf)
fsa_list = ['L9G']

# GUI INPUT
fsa_chosen = "L9G"

years = ['2018', '2019', '2020', '2021', '2022', '2023']
#years = ['2018']

# Jan - 01
# Feb - 02
# Mar - 03
# Apr - 04
# May - 05
# Jun - 06
# Jul - 07
# Aug - 08
# Sep - 09
# Oct - 10
# Nov - 11
# Dec - 12
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

months_name = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

#%% Input files (PRE-PROCESSING)
dirs_inputs = run_student[0]

dirs_hourly_consumption_demand = os.path.join(dirs_inputs, "Hourly_Demand_Data")

###############################################################################
# Dictionary for reading in hourly consumption by FSA
###############################################################################
# FSA -> Year -> Month -> Value
hourly_consumption_data_dic_by_month = {}

for fsa in fsa_list:
    hourly_consumption_data_dic_by_month[fsa] = {} # Initialize FSA dictionary
    for year in years:
        hourly_consumption_data_dic_by_month[fsa][year] = {} # Initialize yearly dictionary
        
        for month in months:
            hourly_consumption_data_dic_by_month[fsa][year][month] = {} # Initialize monthly dictionary
            
            # Initialize dataframes to be used
            hourly_data_date = pd.DataFrame()
            hourly_data_res = pd.DataFrame()
            hourly_data_res_fsa = pd.DataFrame()
            hourly_data_hour_sum = pd.DataFrame()
            
            hourly_data_string = "PUB_HourlyConsumptionByFSA_"+year+month+"_v1.csv"
            
            # Use try and catch if problems reading input data
            try:
                # Not cooked yet, we are going to let it COOK below
                file_path = os.path.join(dirs_hourly_consumption_demand, hourly_data_string)
                hourly_data_raw = pd.read_csv(file_path, skiprows=3, header = 0, usecols= ['FSA', 'DATE', 'HOUR', 'CUSTOMER_TYPE', 'TOTAL_CONSUMPTION'])
            except FileNotFoundError: # not all months had a file (for example, 2024 only has up to may)
                continue
            except ValueError: # skiprows=x does not match the "normal sequence" of 3. For example, 2023 08 data had a different skip_row value
                hourly_data_raw = pd.read_csv(file_path, skiprows=7, header = 0, usecols= ['FSA', 'DATE', 'HOUR', 'CUSTOMER_TYPE', 'TOTAL_CONSUMPTION'])
       
            # Convert Date into year, month, day
            hourly_data_fix_date = hourly_data_raw
            hourly_data_fix_date['DATE'] = pd.to_datetime(hourly_data_raw['DATE'])
            hourly_data_fix_date['YEAR'] = hourly_data_fix_date['DATE'].dt.year
            hourly_data_fix_date['MONTH'] = hourly_data_fix_date['DATE'].dt.month
            hourly_data_fix_date['DAY'] = hourly_data_fix_date['DATE'].dt.day
            
            # Filter out only residential data
            hourly_data_res = hourly_data_fix_date.loc[hourly_data_fix_date['CUSTOMER_TYPE'] == "Residential"].reset_index(drop=True)
            
            # Then filter out by the fsa
            hourly_data_res_fsa = hourly_data_res.loc[hourly_data_res['FSA'] == fsa].reset_index(drop=True)
            
            # Take the sum if fsa has more than 1 date (this is because there are different pay codes in residential loads)
            hourly_data_hour_sum = hourly_data_res_fsa.groupby(["FSA", "CUSTOMER_TYPE", "YEAR", "MONTH", "DAY", "HOUR", "DATE"]).TOTAL_CONSUMPTION.sum().reset_index()
            
            
            hourly_consumption_data_dic_by_month[fsa][year][month] = hourly_data_hour_sum
            
            
            print(hourly_data_string)


JANNA IS RUNNING!
PUB_HourlyConsumptionByFSA_201801_v1.csv
PUB_HourlyConsumptionByFSA_201802_v1.csv
PUB_HourlyConsumptionByFSA_201803_v1.csv
PUB_HourlyConsumptionByFSA_201804_v1.csv
PUB_HourlyConsumptionByFSA_201805_v1.csv
PUB_HourlyConsumptionByFSA_201806_v1.csv
PUB_HourlyConsumptionByFSA_201807_v1.csv
PUB_HourlyConsumptionByFSA_201808_v1.csv
PUB_HourlyConsumptionByFSA_201809_v1.csv
PUB_HourlyConsumptionByFSA_201810_v1.csv
PUB_HourlyConsumptionByFSA_201811_v1.csv
PUB_HourlyConsumptionByFSA_201812_v1.csv
PUB_HourlyConsumptionByFSA_201901_v1.csv
PUB_HourlyConsumptionByFSA_201902_v1.csv
PUB_HourlyConsumptionByFSA_201903_v1.csv
PUB_HourlyConsumptionByFSA_201904_v1.csv
PUB_HourlyConsumptionByFSA_201905_v1.csv
PUB_HourlyConsumptionByFSA_201906_v1.csv
PUB_HourlyConsumptionByFSA_201907_v1.csv
PUB_HourlyConsumptionByFSA_201908_v1.csv
PUB_HourlyConsumptionByFSA_201909_v1.csv
PUB_HourlyConsumptionByFSA_201910_v1.csv
PUB_HourlyConsumptionByFSA_201911_v1.csv
PUB_HourlyConsumptionByFSA_201912_v1.cs

In [ ]:
print(hourly_data_res)
output_hourly_data_path = os.path.join(dirs_inputs, "hourly_data_hour.csv")
hourly_data_res.to_csv(output_hourly_data_path, index=False)

        FSA       DATE  HOUR CUSTOMER_TYPE  TOTAL_CONSUMPTION  YEAR  MONTH  \
0       M1V 2023-12-01     1   Residential             1152.6  2023     12   
1       N6P 2023-12-01     1   Residential             3351.6  2023     12   
2       M2K 2023-12-01     1   Residential              671.3  2023     12   
3       M1R 2023-12-01     1   Residential             4857.5  2023     12   
4       M9V 2023-12-01     1   Residential              162.3  2023     12   
...     ...        ...   ...           ...                ...   ...    ...   
846745  K0C 2023-12-31    24   Residential             1124.1  2023     12   
846746  L7S 2023-12-31    24   Residential             2285.5  2023     12   
846747  N8W 2023-12-31    24   Residential              588.0  2023     12   
846748  M6B 2023-12-31    24   Residential              363.0  2023     12   
846749  N3L 2023-12-31    24   Residential             6534.5  2023     12   

        DAY  
0         1  
1         1  
2         1  
3      

In [11]:
# Group by FSA, DATE, and HOUR, then sum the TOTAL_CONSUMPTION
hourly_data_hour_sum = hourly_data_res.groupby(["FSA", "DATE", "HOUR"]).agg(
    TOTAL_CONSUMPTION=('TOTAL_CONSUMPTION', 'sum')
).reset_index()

# Output the hourly_data_hour_sum to a CSV file
output_hourly_data_path = os.path.join(dirs_inputs, "hourly_data_hour_sum.csv")
hourly_data_hour_sum.to_csv(output_hourly_data_path, index=False)

print(f"Hourly data summary saved to {output_hourly_data_path}")


Hourly data summary saved to ../data\hourly_data_hour_sum.csv


In [14]:
# Group by FSA and calculate min, max, and mean of TOTAL_CONSUMPTION
fsa_summary = hourly_data_hour_sum.groupby("FSA").agg(
    min_consumption=('TOTAL_CONSUMPTION', 'min'),
    max_consumption=('TOTAL_CONSUMPTION', 'max'),
    avg_consumption=('TOTAL_CONSUMPTION', 'mean')
).reset_index()

# Output the FSA summary to a CSV file
output_fsa_summary_path = os.path.join(dirs_inputs, "FSA_consumption_summary.csv")
fsa_summary.to_csv(output_fsa_summary_path, index=False)

print(f"FSA summary statistics saved to {output_fsa_summary_path}")

FSA summary statistics saved to ../data\FSA_consumption_summary.csv


In [17]:
# Path to the input CSV file
output_fsa_summary_path = os.path.join(dirs_inputs, "FSA_consumption_summary.csv")  # Existing path
# Path to the output CSV file
output_cleaned_path = os.path.join(dirs_inputs, "FSA_consumption_summary_cleaned.csv")  # New path for cleaned data

# Step 1: Read the CSV file into a DataFrame
df = pd.read_csv(output_fsa_summary_path)

# Step 2: Remove apostrophes from all string entries
df = df.applymap(lambda x: x.replace("'", "") if isinstance(x, str) else x)

# Step 3: Save the cleaned DataFrame back to a new CSV file
df.to_csv(output_cleaned_path, index=False)

print(f"Cleaned data saved to {output_cleaned_path}")

Cleaned data saved to ../data\FSA_consumption_summary_cleaned.csv


C:\Users\janna\AppData\Local\Temp\ipykernel_10112\1958459033.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace("'", "") if isinstance(x, str) else x)
